Test #1. Pre-Trained BERT Classifier / Misspelled Text

In [15]:
#!pip install pytorch-transformers
#!pip install nlpaug
#!git clone https://github.com/joseph1723/CS376_Final_Project.git

In [16]:
import nlpaug.augmenter.word as naw
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F

In [17]:
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model_spelling = BertModel.from_pretrained('murali1996/bert-base-cased-spell-correction')
tokenizer_spelling = BertTokenizer.from_pretrained('murali1996/bert-base-cased-spell-correction')
if torch.cuda.is_available() :
  print("CUDA")
  device = torch.device("cuda")
  model.to(device)
else :
  print("CPU")
  device = torch.device("cpu")
  model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

CPU


In [19]:
class TestDataset(Dataset) :
  #Dataset - English/typo-added/labeled
  def __init__(self, df) :
    self.df = df
  
  def __len__(self) :
    return len(self.df)
  
  def __getitem__(self, idx):
    text = self.df.iloc[idx, 0]
    label = self.df.iloc[idx, 1]
    return text, label

Hyperparameters

In [20]:
train_size = 1000
test_size = 100
itr = 1
p_itr = 100
epochs = 5

In [21]:
total_df = pd.read_csv('augmented_data/Dataset_aug_profanity_6758_.csv', sep=',')
total_df.dropna(inplace=True)

#Train Set
train_df = total_df.sample(train_size, random_state=999)
train_df = train_df[["text", "label"]]
train_df["label"] = [1 if i == "nothate" else 0 for i in train_df["label"]]

#Test Set
test_df = total_df.sample(test_size, random_state=999)
test_df = test_df[["text", "label"]]
test_df["label"] = [1 if i == "nothate" else 0 for i in test_df["label"]]


In [22]:
train_dataset = TestDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

Data Augmentation / Generating Typo using nlpaug

In [23]:
def augment_data(aug, dataframe, key, N):
  dataframe_copy = dataframe.copy()
  rows = []
  for i, row in dataframe.iterrows() :
    line = row[key]
    line_augmented = aug.augment(line)
    row.iloc[0] = line_augmented
    row_list = list(row)
    rows.append(row_list)
  rows = pd.DataFrame(rows, columns = ['text', 'label'])
  dataframe_copy = dataframe_copy.append(pd.DataFrame(rows), ignore_index=True)
  return dataframe_copy

In [34]:
import torch.nn as nn
#from transformers import BertModel

class CustomBERTModel(nn.Module):
    def __init__(self):
          super(CustomBERTModel, self).__init__()
          self.bert = model_spelling
          # add your additional layers here, for example a dropout layer followed by a linear classification head
          self.dropout = nn.Dropout(0.3)
          self.out = nn.Linear(768, 2)

    def forward(self, ids=None, mask=None, token_type_ids=None):
            sequence_output, pooled_output = self.bert(
                  ids, 
                  attention_mask=mask,
                  token_type_ids=token_type_ids,
                  return_dict=False
            )
            #print(sequence_output)


            # we apply dropout to the sequence output, tensor has shape (batch_size, sequence_length, 768)
            sequence_output = self.dropout(sequence_output)

            # next, we apply the linear layer. The linear layer (which applies a linear transformation)
            # takes as input the hidden states of all tokens (so seq_len times a vector of size 768, each corresponding to
            # a single token in the input sequence) and outputs 2 numbers (scores, or logits) for every token
            # so the logits are of shape (batch_size, sequence_length, 2)
            logits = self.out(sequence_output)

            return logits

In [35]:
model_custom = CustomBERTModel()

for text, label in test_loader:
    encoded_list = [tokenizer_spelling.encode(t, add_special_tokens=True) for t in text]
    #print(encoded_list)
    logits = model_custom(torch.tensor(encoded_list))
    pred = torch.argmax(F.softmax(logits), dim=1)
    print(pred)

/var/folders/kx/qy_c0smd4c14rc316r9rcq3m0000gn/T/ipykernel_53492/3520144061.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)


tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]])
tensor([[0, 0]

Test & Evaluation

*주의해야하는 것은, 학습 샘플의 인풋이 (batch_size, sequence_length)로 들어간다는 것이다. 따라서 zero-padding을 직접 해줘서 model의 forward에 넣어줘야한다.

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

total_loss = 0
total_len = 0
total_correct = 0
X=1


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        MAX_LEN = max(len(e) for e in encoded_list)
        padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels, return_dict=False)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=X)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0
        itr+=1

In [ ]:
model.eval()

total_loss = 0
total_len = 0
total_correct = 0

for text, label in test_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    MAX_LEN = max(len(e) for e in encoded_list)
    padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels, return_dict=False)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)